# Agent Integrations

## [Autogen](https://docs.agentops.ai/v2/examples/autogen)

- Installation

```bash
uv add "autogen-ext[openai]" -U agentops autogen-agentchat python-dotenv
```

In [1]:
import os
import asyncio
import agentops

from dotenv import load_dotenv
load_dotenv()

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.ui import Console

from IPython.core.error import (
    StdinNotImplementedError,
)

In [2]:
# When initializing AgentOps, you can pass in optional tags to help filter sessions
agentops.init(auto_start_session=False)
tracer = agentops.start_trace(
    trace_name="Microsoft Agent Chat Example", tags=["autogen-chat", "microsoft-autogen", "agentops-example"]
)

🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'
🖇 AgentOps: Session Replay for Microsoft Agent Chat Example trace: https://app.agentops.ai/sessions?trace_id=fb599e5096dd3f479f5c2409c0db2925
🖇 AgentOps: You're on the agentops free plan 🤔


AutoGen will now start automatically tracking

- LLM prompts and completions
- Token usage and costs
- Agent names and actions
- Correspondence between agents
- Tool usage
- Errors

In [3]:
# Define model and API key
model_name = "gpt-4o-mini" # Or "gpt-4o" / "gpt-4-turbo" as per migration guide examples

# Create the model client
model_client = OpenAIChatCompletionClient(model=model_name, api_key=os.getenv("OPENAI_API_KEY"))

# Create the agent that uses the LLM.
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",  # Added system message for clarity
    model_client=model_client,
)

user_proxy_initiator = UserProxyAgent("user_initiator")

In [4]:
async def main():

    termination = MaxMessageTermination(max_messages=4)

    group_chat = RoundRobinGroupChat(
    [user_proxy_initiator, assistant],
    termination_condition=termination
    )

    chat_task = "How can I help you today?"
    print(f"User Initiator: {chat_task}")

    try:
        stream = group_chat.run_stream(task=chat_task)
        await Console(stream)
        agentops.end_trace(tracer, end_state="Success")
    except StdinNotImplementedError:
        print("StdinNotImplementedError: This typically happens in non-interactive environments.")
        print("Skipping interactive part of chat for automation.")
        agentops.end_trace(tracer, end_state="Indeterminate")
    except Exception as e:
        print(f"An error occurred: {e}")
        agentops.end_trace(tracer, end_state="Error")
    finally:
        await model_client.close()

In [5]:
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = None

In [6]:
if loop and loop.is_running():
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())
else:
    asyncio.run(main())

User Initiator: How can I help you today?
---------- TextMessage (user) ----------
How can I help you today?


Enter your response:  Who invented electricity?


---------- TextMessage (user_initiator) ----------
Who invented electricity?
---------- TextMessage (assistant) ----------
Electricity as a natural phenomenon was not invented but rather discovered. However, key figures contributed to our understanding and development of electrical concepts. 

1. **Thales of Miletus** (circa 600 BC) is one of the earliest known researchers of static electricity. He discovered that rubbing amber with fur would attract lightweight objects.

2. **Benjamin Franklin** (1706-1790) is famous for his experiments with electricity, including the kite experiment in 1752, which helped demonstrate that lightning is electrical in nature. 

3. **Alessandro Volta** (1745-1827) developed the first true chemical battery, the voltaic pile, in 1800, enabling a steady flow of electricity.

4. **Michael Faraday** (1791-1867) made significant contributions to the understanding of electromagnetism, including the principle of electromagnetic induction.

5. **Thomas Edison** (1

Enter your response:  Ok


---------- TextMessage (user_initiator) ----------
Ok


🖇 AgentOps: Session Replay for Microsoft Agent Chat Example.session trace: https://app.agentops.ai/sessions?trace_id=fb599e5096dd3f479f5c2409c0db2925


## Math Agent

In [3]:
import os
import asyncio
import agentops

from dotenv import load_dotenv
load_dotenv()

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.ui import Console

from typing import Annotated, Literal

from IPython.core.error import (
    StdinNotImplementedError,
)

In [4]:
# When initializing AgentOps, you can pass in optional tags to help filter sessions
agentops.init(auto_start_session=False)
tracer = agentops.start_trace(
    trace_name="Microsoft Agent Chat Example", tags=["autogen-chat", "microsoft-autogen", "agentops-example"]
)

🖇 AgentOps: Session Replay for Microsoft Agent Chat Example trace: https://app.agentops.ai/sessions?trace_id=f9b301f92d556f31bee1a7065874e60f


In [5]:
# Define model and API key
model_name = "gpt-4-turbo"

# Create the model client
model_client = OpenAIChatCompletionClient(model=model_name,
                                          api_key=os.getenv("OPENAI_API_KEY"),
                                          seed=42,
                                          temperature=0)

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")


In [6]:
async def main():
    assistant = AssistantAgent(
        name="Assistant",
        system_message="You are a helpful AI assistant. You can help with simple calculations. Return 'TERMINATE' when the task is done.",
        model_client=model_client,
        tools=[calculator],
        reflect_on_tool_use=True,
    )

    initial_task_message = "What is (1423 - 123) / 3 + (32 + 23) * 5?"
    print(f"User Task: {initial_task_message}")

    try:
        from autogen_core import CancellationToken

        response = await assistant.on_messages(
            [TextMessage(content=initial_task_message, source="user")], CancellationToken()
        )

        final_response_message = response.chat_message
        if final_response_message:
            print(f"Assistant: {final_response_message.to_text()}")
        else:
            print("Assistant did not provide a final message.")

        agentops.end_trace(tracer, end_state="Success")

    except StdinNotImplementedError:
        print("StdinNotImplementedError: This typically happens in non-interactive environments.")
        agentops.end_trace(tracer, end_state="Indeterminate")
    except Exception as e:
        print(f"An error occurred: {e}")
        agentops.end_trace(tracer, end_state="Error")
    finally:
        await model_client.close()


In [7]:
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = None

if loop and loop.is_running():
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())
else:
    asyncio.run(main())

User Task: What is (1423 - 123) / 3 + (32 + 23) * 5?
Assistant: The results of the calculations are:
- \( 1423 - 123 = 1300 \)
- \( 32 + 23 = 55 \)

Now, let's continue with the remaining operations:
- \( 1300 / 3 \)
- \( 55 \times 5 \)


🖇 AgentOps: Session Replay for Microsoft Agent Chat Example.session trace: https://app.agentops.ai/sessions?trace_id=f9b301f92d556f31bee1a7065874e60f


## [LangChain](https://docs.agentops.ai/v2/examples/langchain)

- Installation

```bash
uv add agentops langchain langchain_openai python-dotenv
```


**Note**: You don’t need a `separate agentops.init()` call; the `LangchainCallbackHandler` initializes the AgentOps client automatically if an API key is provided to it or found in the environment.

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Import the ChatOpenAI class from langchain_openai to use OpenAI's chat models
from langchain_openai import ChatOpenAI

# Import necessary components for building and running agents in LangChain
from langchain.agents import (
    tool,                         # Decorator to define a function as a tool for an agent
    AgentExecutor,                # Executes an agent and manages its lifecycle
    create_openai_tools_agent    # Utility to create an agent that uses OpenAI and tools
)

# Import ChatPromptTemplate to construct prompt templates for language models
from langchain_core.prompts import ChatPromptTemplate

# Import the LangChain callback handler from AgentOps for tracing and monitoring agent execution
from agentops.integration.callbacks.langchain import (
    LangchainCallbackHandler as AgentOpsLangchainCallbackHandler  # Rename for clarity
)


In [3]:
# Create a callback handler for AgentOps to monitor and trace the agent's execution.
# You can use tags to categorize or filter traces in the AgentOps dashboard.
agentops_handler = AgentOpsLangchainCallbackHandler(
    tags=["Langchain Example", "agentops-example"]
)

# Initialize the OpenAI language model (LLM) with the callback handler.
# This LLM will use GPT-3.5-turbo and report all interactions to AgentOps.
llm = ChatOpenAI(
    callbacks=[agentops_handler],  # List of callbacks (in this case, just AgentOps)
    model="gpt-3.5-turbo"          # Choose the OpenAI model
)

# Explicitly assign the callback handler again, in case it’s needed for tools or external hooks.
# This ensures that all relevant components are monitored.
llm.callbacks = [agentops_handler]

# Define the prompt structure for the agent.
# `ChatPromptTemplate` is used to define how messages are composed before being sent to the model.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond only in Spanish."),  # System-level instruction
    ("human", "{input}"),                                                 # Placeholder for user's question/input
    ("placeholder", "{agent_scratchpad}"),                                # Holds tool call history and intermediate reasoning steps
    # ("tool_names", "find_movie")                                       # (Optional) Can be used to define tool hints if needed
])


🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'
🖇 AgentOps: You're on the agentops free plan 🤔


Agents generally use tools. Let’s define a simple tool here. Tool usage is also recorded.

In [4]:
@tool
def find_movie(genre: str) -> str:
    """Find available movies"""
    if genre == "drama":
        return "Dune 2"
    else:
        return "Pineapple Express"


tools = [find_movie]

For each tool, you need to also add the callback handler

In [5]:
for t in tools:
    t.callbacks = [agentops_handler]

Add the tools to our LLM

In [6]:
llm_with_tools = llm.bind_tools([find_movie])

Finally, let’s create our agent! Pass in the callback handler to the agent, and all the actions will be recorded in the AO Dashboard

In [7]:
# Create an OpenAI tools-based agent using the language model, tools, and prompt defined earlier.
# This function automatically wraps your LLM and tools into an agent that can reason and act.
agent = create_openai_tools_agent(
    llm,      # The ChatOpenAI instance (GPT-3.5-turbo) with callback support
    tools,    # A list of tools the agent can use (must be defined elsewhere)
    prompt    # The structured chat prompt template
)

# Create an AgentExecutor to manage the execution lifecycle of the agent.
# This includes sending inputs, handling intermediate steps, and producing final output.
agent_executor = AgentExecutor(
    agent=agent,   # The agent created above
    tools=tools    # The same tools passed to the agent; they will be used during execution
)


In [8]:
agent_executor.invoke({"input": "What comedies are playing?"}, config={"callback": [agentops_handler]})

{'input': 'What comedies are playing?',
 'output': '"Pineapple Express" está disponible para ver. ¿Te gustaría más información sobre esta película?'}

## [LangGraph](https://docs.agentops.ai/v2/examples/langgraph)

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

True

### Setup

In [2]:
# Typing helpers to annotate function parameters and state types
from typing import Annotated, Literal, TypedDict

# Import LangGraph's core class for creating and managing state-based agent workflows
from langgraph.graph import StateGraph, END

# Helper function to append messages to a LangGraph state
from langgraph.graph.message import add_messages

# Import the OpenAI chat model for use with LangChain
from langchain_openai import ChatOpenAI

# Message types used in LangChain for structured LLM conversations
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# Decorator to turn Python functions into usable LangChain tools
from langchain_core.tools import tool

# Import the AgentOps library for tracing and monitoring agent behavior
import agentops

# --- AgentOps Initialization ---

# Initialize AgentOps with your API key.
# auto_start_session=False means you'll manually control when tracing begins and ends.
agentops.init(
    os.getenv("AGENTOPS_API_KEY"),  # Securely load your API key from environment variables
    auto_start_session=False        # Avoid auto-starting a trace on import
)

# Manually start a trace session in AgentOps for this specific execution.
# The trace is named "langgraph_example" and will track all LangChain + LangGraph operations.
trace = agentops.start_trace("langgraph_example")


🖇 AgentOps: Session Replay for langgraph_example trace: https://app.agentops.ai/sessions?trace_id=2c2bcb784b4a2c0bb4677fdaa0b15272
🖇 AgentOps: You're on the agentops free plan 🤔


### Define Tools

In [3]:
# Let’s create some simple tools that our agent can use:

@tool
def get_weather(location: str) -> str:
    """Get the weather for a given location."""
    # Simulated weather data
    weather_data = {
        "New York": "Sunny, 72°F",
        "London": "Cloudy, 60°F", 
        "Tokyo": "Rainy, 65°F",
        "Paris": "Partly cloudy, 68°F",
        "Sydney": "Clear, 75°F"
    }
    return weather_data.get(location, f"Weather data not available for {location}")

@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        result = eval(expression)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error calculating expression: {str(e)}"

# Collect tools for binding to the model
tools = [get_weather, calculate]

### Define Agent State

In [4]:
# In LangGraph, we need to define the state that will be passed between nodes:

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

### Create the Model and Node Functions

In [5]:
# --- Create the model with tool binding ---

# Create a ChatOpenAI model instance with deterministic behavior (temperature=0)
# We use the lightweight `gpt-4o-mini` model and bind it to the list of tools
# so that the model can "call" those tools during the reasoning process.
model = ChatOpenAI(temperature=0, model="gpt-4o-mini").bind_tools(tools)


# --- Graph Node: Decision Logic ---

def should_continue(state: AgentState) -> Literal["tools", "end"]:
    """
    Decision node: Determine whether to call tools or end the process.

    Args:
        state (AgentState): The current state containing message history.

    Returns:
        "tools" if the last message contains tool calls.
        "end" if no tool calls are present.
    """
    messages = state["messages"]
    last_message = messages[-1]  # Get the most recent LLM message

    # Check if the LLM output includes tool calls (i.e., it wants to use a tool)
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"  # Go to the tools node

    # No tool call detected – finish the conversation
    return "end"


# --- Graph Node: Call the Language Model ---

def call_model(state: AgentState):
    """
    Node that calls the language model with the current message history.

    Args:
        state (AgentState): The current state, including previous messages.

    Returns:
        A dictionary containing the model's response message.
    """
    messages = state["messages"]
    response = model.invoke(messages)  # Send messages to the LLM and get a response
    return {"messages": [response]}   # Wrap the response in a list for message flow


# --- Graph Node: Execute Tools ---

def call_tools(state: AgentState):
    """
    Node that executes tool calls requested by the model.

    Args:
        state (AgentState): The current state, including messages and tool calls.

    Returns:
        A dictionary with ToolMessage(s) containing the tool results.
    """
    messages = state["messages"]
    last_message = messages[-1]  # The LLM response containing tool_calls

    tool_messages = []  # This will store the responses from each tool

    # Process each tool call requested by the LLM
    for tool_call in last_message.tool_calls:
        tool_name = tool_call["name"]   # Name of the requested tool
        tool_args = tool_call["args"]   # Arguments to pass to the tool

        # Find the matching tool and invoke it
        for tool in tools:
            if tool.name == tool_name:
                result = tool.invoke(tool_args)  # Execute the tool with arguments

                # Create a ToolMessage to send back to the LLM as part of the message chain
                tool_messages.append(
                    ToolMessage(
                        content=str(result),         # Result converted to string
                        tool_call_id=tool_call["id"] # Associate with the original tool call ID
                    )
                )
                break  # Stop looking once we've handled the tool

    # Return tool outputs to be appended to the message list
    return {"messages": tool_messages}


### Build the Graph

In [6]:
# --- Step 1: Create the graph ---

# Initialize a LangGraph workflow using a custom state type (AgentState).
# AgentState is expected to be a TypedDict or similar structure that tracks messages.
workflow = StateGraph(AgentState)

# --- Step 2: Add processing nodes ---

# Add a node named "agent" which runs the language model
workflow.add_node("agent", call_model)

# Add a node named "tools" which executes tools the model requests
workflow.add_node("tools", call_tools)

# --- Step 3: Define the starting point ---

# Set the first node to run when the workflow starts
workflow.set_entry_point("agent")

# --- Step 4: Define conditional routing ---

# After the "agent" node runs, use `should_continue` to decide where to go next
workflow.add_conditional_edges(
    "agent",          # Source node
    should_continue,  # Function to evaluate and determine which edge to follow
    {
        "tools": "tools",  # If model wants tools, go to "tools" node
        "end": END         # Otherwise, end the workflow
    }
)

# --- Step 5: Add loop edge ---

# After executing tools, go back to the agent for further reasoning
workflow.add_edge("tools", "agent")

# --- Step 6: Compile the graph into an executable app ---

# Once all nodes and edges are set up, compile the workflow into an app
# This app can now be invoked with input state to run the full reasoning/tool-use loop
app = workflow.compile()


### Run Examples

In [7]:
# Example 1: Weather query
print("Example 1: Weather Query")
print("=" * 50)

messages = [HumanMessage(content="What's the weather in New York and Tokyo?")]
result = app.invoke({"messages": messages})

final_message = result["messages"][-1]
print(f"Response: {final_message.content}\n")

Example 1: Weather Query
Response: The weather is as follows:
- **New York**: Sunny, 72°F
- **Tokyo**: Rainy, 65°F



In [8]:
# Example 2: Math calculation
print("Example 2: Math Calculation")
print("=" * 50)

messages = [HumanMessage(content="Calculate 25 * 4 + 10")]
result = app.invoke({"messages": messages})

final_message = result["messages"][-1]
print(f"Response: {final_message.content}\n")

Example 2: Math Calculation
Response: The result of the calculation \( 25 \times 4 + 10 \) is 110.



In [9]:
# Example 3: Combined query
print("Example 3: Combined Query")
print("=" * 50)

messages = [HumanMessage(content="What's the weather in Paris? Also calculate 100/5")]
result = app.invoke({"messages": messages})

final_message = result["messages"][-1]
print(f"Response: {final_message.content}\n")

Example 3: Combined Query
Response: The weather in Paris is partly cloudy with a temperature of 68°F. Additionally, the result of the calculation \(100 \div 5\) is 20.0.



In [10]:
print("✅ Check your AgentOps dashboard for comprehensive traces!")
print("🔍 You'll see the graph structure, execution flow, and all LLM/tool calls.")
agentops.end_trace(trace)

✅ Check your AgentOps dashboard for comprehensive traces!
🔍 You'll see the graph structure, execution flow, and all LLM/tool calls.


🖇 AgentOps: Session Replay for langgraph_example.session trace: https://app.agentops.ai/sessions?trace_id=2c2bcb784b4a2c0bb4677fdaa0b15272


## [Smolagents](https://docs.agentops.ai/v2/integrations/smolagents)

### Core Concepts

Smolagents is designed around several key concepts:

- **Agents**: AI assistants that can use tools and reason through problems
- **Tools**: Functions that agents can call to interact with external systems
- **Models**: LLM backends that power agent reasoning (supports various providers via LiteLLM)
-**Code Execution**: Agents can write and execute Python code in sandboxed environments
- **Multi-Agent Systems**: Orchestrate multiple specialized agents working together

### Installation

Install AgentOps and Smolagents, along with any additional dependencies:

```
uv add agentops 'smolagents[litellm]' python-dotenv duckduckgo-search
```

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import agentops
from smolagents import LiteLLMModel, ToolCallingAgent, DuckDuckGoSearchTool

# Initialize AgentOps
agentops.init()

# Create a model (supports various providers via LiteLLM)
model = LiteLLMModel("openai/gpt-4o-mini")

# Create an agent with tools
agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
)

# Run the agent
result = agent.run("What are the latest developments in AI safety research?")
print(result)

🖇 AgentOps: SmoLAgents instrumentation enabled
/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/smolagents/default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What are the latest developments in AI safety research?                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content=None, rol...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='to...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content=None, rol...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - seria

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'latest developments in AI safety research 2023'}          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Observations: Currently, I couldn't find specific details about recent advancements in AI safety research for 2023 
from the search results. I recommend checking reputable journals or news sources focused on AI for the latest 
updates.

Final answer: Currently, I couldn't find specific details about recent advancements in AI safety research for 2023 
from the search results. I recommend checking reputable journals or news sources focused on AI for the latest 
updates.

[Step 13: Duration 1.67 seconds| Input tokens: 87,479 | Output tokens: 301]

Currently, I couldn't find specific details about recent advancements in AI safety research for 2023 from the search results. I recommend checking reputable journals or news sources focused on AI for the latest updates.


/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/aiohttp/connector.py:963: DeprecationWarning: enable_cleanup_closed ignored because https://github.com/python/cpython/pull/118960 is fixed in Python version sys.version_info(major=3, minor=12, micro=11, releaselevel='final', serial=0)
  super().__init__(
🖇 AgentOps: You're on the agentops free plan 🤔


Simple Math Agent

In [1]:
import agentops
from smolagents import LiteLLMModel, CodeAgent

# Initialize AgentOps
agentops.init()

# Create a model
model = LiteLLMModel("openai/gpt-4o-mini")

# Create a code agent that can perform calculations
agent = CodeAgent(
    tools=[],  # No external tools needed for math
    model=model,
    additional_authorized_imports=["math", "numpy"],
)

# Ask the agent to solve a math problem
result = agent.run(
    "Calculate the compound interest on $10,000 invested at 5% annual rate "
    "for 10 years, compounded monthly. Show your work."
)

print(result)

🖇 AgentOps: SmoLAgents instrumentation enabled


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the compound interest on $10,000 invested at 5% annual rate for 10 years, compounded monthly. Show    │
│ your work.                                                                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  P = 10000  # Principal amount                                                                                    
  r = 0.05   # Annual interest rate                                                                                
  n = 12     # Compounding frequency (monthly)                                                                     
  t = 10     # Duration in years                                                                                   
                                                                                                                   
  # Calculate total amount after interest                                                                          
  A = P * (1 + r/n)**(n*t)                                                                                         
  compound_interest = A - P                                                                                        
  compound_interest                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: 6470.094976902801

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(compound_interest)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 6470.094976902801

[Step 2: Duration 2.61 seconds| Input tokens: 4,245 | Output tokens: 361]

6470.094976902801


🖇 AgentOps: You're on the agentops free plan 🤔
/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content=None, rol...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='to...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content=None, rol...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnex

Simple research agent

In [2]:
import agentops
from smolagents import (
    LiteLLMModel,
    ToolCallingAgent,
    DuckDuckGoSearchTool,
    tool
)

# Initialize AgentOps
agentops.init()

# Create a custom tool
@tool
def word_counter(text: str) -> str:
    """
    Counts the number of words in a given text.
    
    Args:
        text: The text to count words in.
        
    Returns:
        A string with the word count.
    """
    word_count = len(text.split())
    return f"The text contains {word_count} words."

# Create model and agent
model = LiteLLMModel("openai/gpt-4o-mini")

agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), word_counter],
    model=model,
)

# Run a research task
result = agent.run(
    "Search for information about the James Webb Space Telescope's latest discoveries. "
    "Then count how many words are in your summary."
)

print(result)

/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/smolagents/default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for information about the James Webb Space Telescope's latest discoveries. Then count how many words are │
│ in your summary.                                                                                                │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'James Webb Space Telescope latest discoveries'}           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|对一个陌生的英文名字，如何快速确定哪个是姓哪个是名？ - 知乎](https://www.zhihu.com/question/26601306)
知乎，中文互联网高质量的问答社区和创作者聚集的原创内容平台，于 2011 年 1 
月正式上线，以「让人们更好的分享知识、经验和见解，找到自己的解答」为品牌使命。知乎凭借认真、专业 …

|英文原版教材中，托马斯微积分和James Stewart编的微积分有什 
…](https://www.zhihu.com/question/25582114/answers/updated)
英文原版教材中，托马斯微积分和James Stewart编的微积分有什么各自的特点？ 
两本教材都是传世之作，Thomas版有60多年的历史，Thomas老先生90岁依然奋战在MIT教学第一 …

|勒布朗·詹姆斯（LeBron James） - 知乎](https://www.zhihu.com/topic/19655352)
勒布朗·詹姆斯（LeBron 
James），1984年12月30日出生于美国俄亥俄州阿克伦（Akron,Ohio），美国职业篮球运动员，司职小前锋，绰号“小皇帝”，效力于
NBA洛杉矶湖人 …

|如何评价诺贝尔生理学或医学奖得主、DNA 之父詹姆斯·沃森 …](https://www.zhihu.com/tardis/bd/ans/1378066318)
詹姆斯·杜威·沃森（James Dewey 
Watson）1953年和克里克发现DNA双螺旋结构（包括中心法则），1962年，沃森与克里克，以及威尔金斯共享诺贝尔奖，DNA双螺旋
结构的发现是20世纪 …

|红皮书，白皮书，蓝皮书有什么区别？ - 知乎](https://www.zhihu.com/question/47756700)
图片来源：官网截图. 此外我们也会看到这些皮书：《中国民营银行蓝皮书》（2017 
罗兰贝格）、《中国汽车品牌及新能源汽车发展趋势蓝皮书》（2019 巨量引擎·汽车数据策略研究院）、 …

|如何评价电影配乐家詹姆斯·霍纳（James Horner）？ - 知乎](https://www.zhihu.com/question/22172948)
当年我的第一张电影音乐专辑就是我爸给我买的霍纳精选集；整整一周前，我还在上海电影节看的Southpaw，没想到就成了他的遗
作。尽管新世纪以来James Horner的才华似乎慢慢变少了， …

|为什么流行的歌里都提及James Dean？ - 知乎](https://www.zhihu.com/question/30016354)
可见，James Dean在美国大众文化中，最具特色的是他的眼睛。他的眼睛是怎样的呢？和好友玉婆一样的紫罗兰色？答主不知。 
再回到第一例，为什么James Dean是垮掉一代的代表，却作 …

|怎样理解亨利·詹姆斯的《真东西》？ - 知乎](https://www.zhihu.com/question/647068843)
知乎，中文互联网高质量的问答社区和创作者聚集的原创内容平台，于 2011 年 1 
月正式上线，以「让人们更好的分享知识、经验和见解，找到自己的解答」为品牌使命。知乎凭借认真、专业 …

|如何评价why nations fail的中文版? - 知乎](https://www.zhihu.com/question/31265291)
当然了，Daron Acemoglu 和James A. 
Robinson也不是吃素，写了一篇言辞较为激烈的回应，然后Sachs又针对回应写了回应，不过这都是后话了。 
不论怎样，Sachs的文章从非常专业的角度 …

|梅奔车队 it's james是什么梗呀？ - 知乎](https://www.zhihu.com/question/353050081)
Oct 29, 2019 · 如题 有人科普吗？有人说是车队鼓励师 有人说是要让车时james就出来了

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The James Webb Space Telescope (JWST) has made         │
│ significant discoveries in 2023. It has captured images of distant galaxies, providing new insights into the    │
│ formation of stars and planets. JWST has also been instrumental in analyzing the atmospheres of exoplanets,     │
│ revealing key information about their composition and potential habitability. One of its most exciting findings │
│ includes detecting organic molecules in the atmosphere of a distant exoplanet, suggesting the building blocks   │
│ of life may exist beyond our solar system. Additionally, the telescope is helping astronomers understand galaxy │
│ evolution over cosmic time, showcasing the early stages of galaxy formation and the role of supermassive black  │
│ holes. The text contains 104 words.'}                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: The James Webb Space Telescope (JWST) has made significant discoveries in 2023. It has captured 
images of distant galaxies, providing new insights into the formation of stars and planets. JWST has also been 
instrumental in analyzing the atmospheres of exoplanets, revealing key information about their composition and 
potential habitability. One of its most exciting findings includes detecting organic molecules in the atmosphere of
a distant exoplanet, suggesting the building blocks of life may exist beyond our solar system. Additionally, the 
telescope is helping astronomers understand galaxy evolution over cosmic time, showcasing the early stages of 
galaxy formation and the role of supermassive black holes. The text contains 104 words.

Final answer: The James Webb Space Telescope (JWST) has made significant discoveries in 2023. It has captured 
images of distant galaxies, providing new insights into the formation of stars and planets. JWST has also been 
instrumental in analyzing the atmospheres of exoplanets, revealing key information about their composition and 
potential habitability. One of its most exciting findings includes detecting organic molecules in the atmosphere of
a distant exoplanet, suggesting the building blocks of life may exist beyond our solar system. Additionally, the 
telescope is helping astronomers understand galaxy evolution over cosmic time, showcasing the early stages of 
galaxy formation and the role of supermassive black holes. The text contains 104 words.

[Step 14: Duration 4.07 seconds| Input tokens: 105,263 | Output tokens: 576]

The James Webb Space Telescope (JWST) has made significant discoveries in 2023. It has captured images of distant galaxies, providing new insights into the formation of stars and planets. JWST has also been instrumental in analyzing the atmospheres of exoplanets, revealing key information about their composition and potential habitability. One of its most exciting findings includes detecting organic molecules in the atmosphere of a distant exoplanet, suggesting the building blocks of life may exist beyond our solar system. Additionally, the telescope is helping astronomers understand galaxy evolution over cosmic time, showcasing the early stages of galaxy formation and the role of supermassive black holes. The text contains 104 words.


Simple Multi-steps agent

In [4]:
import agentops
from smolagents import LiteLLMModel, CodeAgent, tool
import json

# Initialize AgentOps
agentops.init()

# Create tools for data processing
@tool
def save_json(data: dict, filename: str) -> str:
    """
    Saves data to a JSON file.
    
    Args:
        data: Dictionary to save
        filename: Name of the file to save to
        
    Returns:
        Success message
    """
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
    return f"Data saved to {filename}"

@tool
def load_json(filename: str) -> dict:
    """
    Loads data from a JSON file.
    
    Args:
        filename: Name of the file to load from
        
    Returns:
        The loaded data as a dictionary
    """
    with open(filename, 'r') as f:
        return json.load(f)

# Create agent
model = LiteLLMModel("openai/gpt-4o-mini")

agent = CodeAgent(
    tools=[save_json, load_json],
    model=model,
    additional_authorized_imports=["pandas", "datetime"],
)

# Run a multi-step data processing task
result = agent.run("""
1. Create a dataset of 5 fictional employees with names, departments, and salaries
2. Save this data to 'employees.json'
3. Load the data back and calculate the average salary
4. Find the highest paid employee
5. Return a summary of your findings
""")

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 1. Create a dataset of 5 fictional employees with names, departments, and salaries                              │
│ 2. Save this data to 'employees.json'                                                                           │
│ 3. Load the data back and calculate the average salary                                                          │
│ 4. Find the highest paid employee                                                                               │
│ 5. Return a summary of your findings                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  employees = [                                                                                                    
      {"name": "Alice Johnson", "department": "Engineering", "salary": 75000},                                     
      {"name": "Bob Smith", "department": "Marketing", "salary": 60000},                                           
      {"name": "Charlie Brown", "department": "Sales", "salary": 55000},                                           
      {"name": "Diana Prince", "department": "HR", "salary": 70000},                                               
      {"name": "Ethan Hunt", "department": "Finance", "salary": 80000}                                             
  ]                                                                                                                
                                                                                                                   
  save_json(data=employees, filename='employees.json')                                                             
  print("Dataset of fictional employees created and saved.")                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Dataset of fictional employees created and saved.

Out: None

[Step 1: Duration 5.76 seconds| Input tokens: 2,025 | Output tokens: 272]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  summary = {                                                                                                      
      "average_salary": average_salary,                                                                            
      "highest_paid_employee": highest_paid_employee                                                               
  }                                                                                                                
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'average_salary': 68000.0, 'highest_paid_employee': {'name': 'Ethan Hunt', 'department': 'Finance', 
'salary': 80000}}

[Step 4: Duration 1.92 seconds| Input tokens: 10,445 | Output tokens: 530]

{'average_salary': 68000.0, 'highest_paid_employee': {'name': 'Ethan Hunt', 'department': 'Finance', 'salary': 80000}}


## Orchestrate a Multi-Agent System

In this notebook, we will make a multi-agent web browser: an agentic system with several agents collaborating to solve problems using the web! It will be a simple hierarchy, using a `ManagedAgent` object to wrap the managed web search agent:

```
+----------------+
| Manager agent  |
+----------------+
         |
_________|______________
|                        |
Code interpreter   +--------------------------------+
       tool        |         Managed agent          |
                   |      +------------------+      |
                   |      | Web Search agent |      |
                   |      +------------------+      |
                   |         |            |         |
                   |  Web Search tool     |         |
                   |             Visit webpage tool |
                   +--------------------------------+
```

Let’s set up this system. Run the line below to install the required dependencies:

```
uv add agentops duckduckgo-search markdownify smolagents
```

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import agentops
import os
import re
import requests
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import LiteLLMModel, tool, CodeAgent, ToolCallingAgent, DuckDuckGoSearchTool

⚡️ Our agent will be powered by openai/gpt-4o-mini using the LiteLLMModel class.

In [3]:
from smolagents import LiteLLMModel, tool ,CodeAgent, ToolCallingAgent, DuckDuckGoSearchTool
agentops.init(auto_start_session=False)
tracer = agentops.start_trace(
    trace_name="Orchestrate a Multi-Agent System", tags=["smolagents", "example", "multi-agent", "agentops-example"]
)
model = LiteLLMModel("openai/gpt-4o-mini")

🖇 AgentOps: SmoLAgents instrumentation enabled
🖇 AgentOps: Session Replay for Orchestrate a Multi-Agent System trace: https://app.agentops.ai/sessions?trace_id=e60c132e98705340d53269c6cd0942fa
🖇 AgentOps: You're on the agentops free plan 🤔


### Create a Web Search Tool

For web browsing, we can already use our pre-existing `DuckDuckGoSearchTool`. However, we will also create a `VisitWebpageTool` from scratch using `markdownify`. Here’s how:

In [5]:
@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [6]:
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[:500])

Hugging Face - Wikipedia

[Jump to content](#bodyContent)

Main menu

Main menu

move to sidebar
hide

Navigation

* [Main page](/wiki/Main_Page "Visit the main page [z]")
* [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")
* [Current events](/wiki/Portal:Current_events "Articles related to current events")
* [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")
* [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")
* [Conta


### Build Our Multi-Agent System
We will now use the tools `search` and `visit_webpage` to create the web agent.

In [7]:
web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],
    model=model,
    name="search",
    description="Runs web searches for you. Give it your query as an argument.",
)

manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent],
    additional_authorized_imports=["time", "numpy", "pandas"],
)

/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/smolagents/default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)


Let’s run our system with the following query:

In [8]:
answer = manager_agent.run(
    "If LLM trainings continue to scale up at the current rhythm until 2030, what would be the electric power in GW required to power the biggest training runs by 2030? What does that correspond to, compared to some countries? Please provide a source for any number used."
)

print(answer)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ If LLM trainings continue to scale up at the current rhythm until 2030, what would be the electric power in GW  │
│ required to power the biggest training runs by 2030? What does that correspond to, compared to some countries?  │
│ Please provide a source for any number used.                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Thought:...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Thought:...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - seria

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Conducting a web search to find current electric power consumption for LLM trainings and projections.          
  search(task="Find current electric power consumption for large language model trainings and projections until    
  2030, including relevant sources.", additional_args={})                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────────────── New run - search ────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'search'.                                                                          │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Find current electric power consumption for large language model trainings and projections until 2030,          │
│ including relevant sources.                                                                                     │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {}.                                                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────╯

...

/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content="Thought:...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/home/locch/Works/zsrc/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content="Thought:...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - seria

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(final_summary)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'Projected Power for LLMs in 2030 (GW)': 327, 'Comparison with Major Countries': {'China': 
1026.255707762557, 'United States': 473.17351598173514, 'India': 174.88584474885846, 'Russia': 116.89497716894977, 
'Japan': 104.22374429223744, 'Brazil': 73.63013698630137, 'Canada': 64.38356164383562, 'South Korea': 
63.6986301369863, 'Germany': 52.96803652968037, 'France': 46.80365296803653}}

[Step 5: Duration 5.46 seconds| Input tokens: 17,968 | Output tokens: 1,443]

{'Projected Power for LLMs in 2030 (GW)': 327, 'Comparison with Major Countries': {'China': 1026.255707762557, 'United States': 473.17351598173514, 'India': 174.88584474885846, 'Russia': 116.89497716894977, 'Japan': 104.22374429223744, 'Brazil': 73.63013698630137, 'Canada': 64.38356164383562, 'South Korea': 63.6986301369863, 'Germany': 52.96803652968037, 'France': 46.80365296803653}}


Awesome! We’ve successfully run a multi-agent system. Let’s end the agentops session with a “Success” state. You can also end the session with a “Failure” or “Indeterminate” state, which is set as default.

In [9]:
agentops.end_trace(tracer, end_state="Success")

🖇 AgentOps: Session Replay for Orchestrate a Multi-Agent System.session trace: https://app.agentops.ai/sessions?trace_id=e60c132e98705340d53269c6cd0942fa


## [OpenAI Agents](https://docs.agentops.ai/v2/examples/openai_agents)

### Airline Customer Service Agent

This is a simple chatbot designed to assist airline customers with common queries. Here the agents are also used as tools to help the bot answer questions more effectively. Using AgentOps we can track the flow of the conversation and the agents used. This is useful for debugging and understanding how the bot is performing.

In [1]:
# Set the API keys for your AgentOps and OpenAI accounts.
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["AGENTOPS_API_KEY"] = os.getenv("AGENTOPS_API_KEY", "your_api_key_here")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "your_openai_api_key_here")

In [2]:
from __future__ import annotations as _annotations  # noqa: F404

import random
import uuid

from pydantic import BaseModel
import agentops

from agents import (  # noqa: E402
    Agent,
    HandoffOutputItem,
    ItemHelpers,
    MessageOutputItem,
    RunContextWrapper,
    Runner,
    ToolCallItem,
    ToolCallOutputItem,
    TResponseInputItem,
    function_tool,
    handoff,
    trace,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX  # noqa: E402

In [3]:
agentops.init(tags=["customer-service-agent", "openai-agents", "agentops-example"])
tracer = agentops.start_trace(trace_name="Customer Service Agent")

🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'
🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'
🖇 AgentOps: Session Replay for Customer Service Agent trace: https://app.agentops.ai/sessions?trace_id=59d762052a959d0f14e299314b92d886
🖇 AgentOps: You're on the agentops free plan 🤔


In [4]:
# Context model for the airline agent
class AirlineAgentContext(BaseModel):
    passenger_name: str | None = None
    confirmation_number: str | None = None
    seat_number: str | None = None
    flight_number: str | None = None

In [5]:
# Tools for the airline agent
@function_tool(name_override="faq_lookup_tool", description_override="Lookup frequently asked questions.")
async def faq_lookup_tool(question: str) -> str:
    if "bag" in question or "baggage" in question:
        return (
            "You are allowed to bring one bag on the plane. "
            "It must be under 50 pounds and 22 inches x 14 inches x 9 inches."
        )
    elif "seats" in question or "plane" in question:
        return (
            "There are 120 seats on the plane. "
            "There are 22 business class seats and 98 economy seats. "
            "Exit rows are rows 4 and 16. "
            "Rows 5-8 are Economy Plus, with extra legroom. "
        )
    elif "wifi" in question:
        return "We have free wifi on the plane, join Airline-Wifi"
    return "I'm sorry, I don't know the answer to that question."


@function_tool
async def update_seat(context: RunContextWrapper[AirlineAgentContext], confirmation_number: str, new_seat: str) -> str:
    """
    Update the seat for a given confirmation number.

    Args:
        confirmation_number: The confirmation number for the flight.
        new_seat: The new seat to update to.
    """
    # Update the context based on the customer's input
    context.context.confirmation_number = confirmation_number
    context.context.seat_number = new_seat
    # Ensure that the flight number has been set by the incoming handoff
    assert context.context.flight_number is not None, "Flight number is required"
    return f"Updated seat to {new_seat} for confirmation number {confirmation_number}"


### HOOKS

async def on_seat_booking_handoff(context: RunContextWrapper[AirlineAgentContext]) -> None:
    flight_number = f"FLT-{random.randint(100, 999)}"
    context.context.flight_number = flight_number


### AGENTS

faq_agent = Agent[AirlineAgentContext](
    name="FAQ Agent",
    handoff_description="A helpful agent that can answer questions about the airline.",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    You are an FAQ agent. If you are speaking to a customer, you probably were transferred to from the triage agent.
    Use the following routine to support the customer.
    # Routine
    1. Identify the last question asked by the customer.
    2. Use the faq lookup tool to answer the question. Do not rely on your own knowledge.
    3. If you cannot answer the question, transfer back to the triage agent.""",
    tools=[faq_lookup_tool],
)

seat_booking_agent = Agent[AirlineAgentContext](
    name="Seat Booking Agent",
    handoff_description="A helpful agent that can update a seat on a flight.",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    You are a seat booking agent. If you are speaking to a customer, you probably were transferred to from the triage agent.
    Use the following routine to support the customer.
    # Routine
    1. Ask for their confirmation number.
    2. Ask the customer what their desired seat number is.
    3. Use the update seat tool to update the seat on the flight.
    If the customer asks a question that is not related to the routine, transfer back to the triage agent. """,
    tools=[update_seat],
)

triage_agent = Agent[AirlineAgentContext](
    name="Triage Agent",
    handoff_description="A triage agent that can delegate a customer's request to the appropriate agent.",
    instructions=(
        f"{RECOMMENDED_PROMPT_PREFIX} "
        "You are a helpful triaging agent. You can use your tools to delegate questions to other appropriate agents."
    ),
    handoffs=[
        faq_agent,
        handoff(agent=seat_booking_agent, on_handoff=on_seat_booking_handoff),
    ],
)

In [6]:
faq_agent.handoffs.append(triage_agent)
seat_booking_agent.handoffs.append(triage_agent)

In [7]:
async def main():
    current_agent: Agent[AirlineAgentContext] = triage_agent
    input_items: list[TResponseInputItem] = []
    context = AirlineAgentContext()

    # Normally, each input from the user would be an API request to your app, and you can wrap the request in a trace()
    # Here, we'll just use a random UUID for the conversation ID
    conversation_id = uuid.uuid4().hex[:16]

    while True:
        user_input = input("Enter your message: ")
        with trace("Customer service", group_id=conversation_id):
            input_items.append({"content": user_input, "role": "user"})
            result = await Runner.run(current_agent, input_items, context=context)

            for new_item in result.new_items:
                agent_name = new_item.agent.name
                if isinstance(new_item, MessageOutputItem):
                    print(f"{agent_name}: {ItemHelpers.text_message_output(new_item)}")
                elif isinstance(new_item, HandoffOutputItem):
                    print(f"Handed off from {new_item.source_agent.name} to {new_item.target_agent.name}")
                elif isinstance(new_item, ToolCallItem):
                    print(f"{agent_name}: Calling a tool")
                elif isinstance(new_item, ToolCallOutputItem):
                    print(f"{agent_name}: Tool call output: {new_item.output}")
                else:
                    print(f"{agent_name}: Skipping item: {new_item.__class__.__name__}")
            input_items = result.to_input_list()
            current_agent = result.last_agent

In [ ]:
await main()
agentops.end_trace(tracer, end_state="Success")

Enter your message:  Can I bring a bag on the flight?


Triage Agent: Skipping item: HandoffCallItem
Handed off from Triage Agent to FAQ Agent
FAQ Agent: Calling a tool
FAQ Agent: Tool call output: You are allowed to bring one bag on the plane. It must be under 50 pounds and 22 inches x 14 inches x 9 inches.
FAQ Agent: You are allowed to bring one bag on the plane, and it must be under 50 pounds and 22 inches x 14 inches x 9 inches in size.


Enter your message:  How many seats are there in business class?


FAQ Agent: Calling a tool
FAQ Agent: Skipping item: HandoffCallItem
FAQ Agent: Tool call output: There are 120 seats on the plane. There are 22 business class seats and 98 economy seats. Exit rows are rows 4 and 16. Rows 5-8 are Economy Plus, with extra legroom. 
Handed off from FAQ Agent to Triage Agent
Triage Agent: There are 22 seats in business class.


Enter your message:  Success


Triage Agent: Great! If you have any more questions, feel free to ask. Safe travels!
